# Efficient Yelp API Calls
---

Author: Jackson Muehlbauer

Date: 2/27/2023

In [1]:
# imports
import json
import pandas as pd
from yelpapi import YelpAPI
import os, json, math, time
from tqdm.notebook import tqdm_notebook

In [2]:
with open('/Users/jacksonmuehlbauer/.secret/yelp_api.json') as f:
    login = json.load(f)

login.keys()

dict_keys(['client-id', 'api-key'])

In [3]:
# make yelp_api variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)
yelp_api

In [4]:
# search conditions
LOCATION = 'Minneapolis, MN'
TERM = 'Ramen'

In [7]:
JSON_FILE = 'Data/minneapolis_ramen.json'

In [8]:
# function from Coding Dojo
def create_json_file(JSON_FILE,  delete_if_exists=False):
    
    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    
    ## If it DOES exist:
    if file_exists == True:
        
        ## Check if user wants to delete if exists
        if delete_if_exists==True:
            
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE)
            ## Recursive call to function after old file deleted
            create_json_file(JSON_FILE,delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")            
            
            
    ## If it does NOT exist:
    else:
        
        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        
        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)
        
        ## If JSON_FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            json.dump([],f)

In [9]:
## Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists = True)

## Laod previous results and use len of results for offset

with open(JSON_FILE, 'r') as f:
    previous_results = json.load(f)
    
# set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

# use out yelp_api variable's search_query method to perform our API call

results = yelp_api.search_query(location = LOCATION, term = TERM, offset = n_results)

## How many results total?
total_results = results['total']

## How many did we got the details for?
results_per_page = len(results['businesses'])

# Use math.ceil to round up for the total number of pages of results
n_pages = math.ceil((results['total']-n_results)/results_per_page)
n_pages

[i] Data/minneapolis_ramen.json not found. Saving empty list to new file.
- 0 previous results found.


12

In [10]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)

  0%|          | 0/12 [00:00<?, ?it/s]

In [11]:
df = pd.read_json(JSON_FILE)
display(df.head(), df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,xoq8XRgV5f031ZI3aHrSFg,ramen-kazama-minneapolis,Ramen Kazama,https://s3-media1.fl.yelpcdn.com/bphoto/Bbal0I...,False,https://www.yelp.com/biz/ramen-kazama-minneapo...,385,"[{'alias': 'ramen', 'title': 'Ramen'}]",4.0,"{'latitude': 44.9410781860352, 'longitude': -9...","[pickup, delivery]",$$,"{'address1': '3400 Nicollet Ave S', 'address2'...",+16123536160,(612) 353-6160,3510.461432
1,94ciL-h9yS6pm0BOFzjLEA,kyuramen-eagan-eagan,Kyuramen - Eagan,https://s3-media1.fl.yelpcdn.com/bphoto/DQXmRt...,False,https://www.yelp.com/biz/kyuramen-eagan-eagan?...,71,"[{'alias': 'ramen', 'title': 'Ramen'}, {'alias...",4.0,"{'latitude': 44.83054, 'longitude': -93.16147}","[restaurant_reservation, delivery, pickup]",$$,"{'address1': '1278 Town Centre Dr', 'address2'...",+19526001032,(952) 600-1032,17431.837488
2,51SWfc0I300IpUZcv7kCfA,zen-box-izakaya-minneapolis,Zen Box Izakaya,https://s3-media2.fl.yelpcdn.com/bphoto/sCMGcA...,False,https://www.yelp.com/biz/zen-box-izakaya-minne...,691,"[{'alias': 'ramen', 'title': 'Ramen'}, {'alias...",4.0,"{'latitude': 44.9784334, 'longitude': -93.2596...","[pickup, delivery]",$$,"{'address1': '602 Washington Ave S', 'address2...",+16123323936,(612) 332-3936,901.681643
3,ivttbmD-kRcpehU6WqOJrA,ramen-kawae-minneapolis,Ramen Kawae,https://s3-media4.fl.yelpcdn.com/bphoto/y9TQqV...,False,https://www.yelp.com/biz/ramen-kawae-minneapol...,12,"[{'alias': 'ramen', 'title': 'Ramen'}]",4.5,"{'latitude': 44.98772, 'longitude': -93.27724}",[],NaN,"{'address1': '729 Washington Ave N', 'address2...",,,2340.875059
4,O-On_JfItCH5lFudaUAS6w,tenka-ramen-minneapolis,Tenka Ramen,https://s3-media1.fl.yelpcdn.com/bphoto/0XEKf4...,False,https://www.yelp.com/biz/tenka-ramen-minneapol...,142,"[{'alias': 'ramen', 'title': 'Ramen'}]",4.5,"{'latitude': 44.948677, 'longitude': -93.297478}","[pickup, delivery]",$$,"{'address1': '1404 W Lake St', 'address2': '',...",+16128254128,(612) 825-4128,3718.596644


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
235,nfq4xl7gm_GWt7BVXjTVIg,winnipeg-grocery-saint-paul,Winnipeg Grocery,,False,https://www.yelp.com/biz/winnipeg-grocery-sain...,2,"[{'alias': 'grocery', 'title': 'Grocery'}]",1.0,"{'latitude': 44.96725, 'longitude': -93.1057951}","[delivery, pickup]",NaN,"{'address1': '864 Rice St', 'address2': None, ...",+16514873110,(651) 487-3110,12260.384052
236,bF9JDTet_wiotfLLolBkSQ,speedway-anoka,Speedway,https://s3-media1.fl.yelpcdn.com/bphoto/u9VKpm...,False,https://www.yelp.com/biz/speedway-anoka?adjust...,2,"[{'alias': 'servicestations', 'title': 'Gas St...",1.0,"{'latitude': 45.224957, 'longitude': -93.398059}",[delivery],NaN,"{'address1': '14000 Saint Francis Blvd', 'addr...",+17635769444,(763) 576-9444,30239.063822
237,_dW0VPsNOknQfI2NolM6IQ,barnes-and-noble-eagan,Barnes & Noble,https://s3-media2.fl.yelpcdn.com/bphoto/ZyIswE...,False,https://www.yelp.com/biz/barnes-and-noble-eaga...,14,"[{'alias': 'bookstores', 'title': 'Bookstores'...",3.0,"{'latitude': 44.8363367696625, 'longitude': -9...",[],$$,"{'address1': '1291 Promenade Pl', 'address2': ...",+16516831955,(651) 683-1955,17025.570656
238,itt2WUcllGAkaaM6z_Kndw,speedway-plymouth-6,Speedway,https://s3-media1.fl.yelpcdn.com/bphoto/t0yyxA...,False,https://www.yelp.com/biz/speedway-plymouth-6?a...,4,"[{'alias': 'servicestations', 'title': 'Gas St...",2.0,"{'latitude': 45.0341, 'longitude': -93.50892}",[delivery],NaN,"{'address1': '4325 Peony Ln N', 'address2': No...",+17634789091,(763) 478-9091,20679.908444
239,rDMLbgP1k0fpRJc8BIWepw,bambu-kitchen-and-sushi-bar-oakdale,Bambu Kitchen & Sushi Bar,https://s3-media3.fl.yelpcdn.com/bphoto/n5FX7N...,False,https://www.yelp.com/biz/bambu-kitchen-and-sus...,67,"[{'alias': 'vietnamese', 'title': 'Vietnamese'...",3.5,"{'latitude': 44.9602497095307, 'longitude': -9...","[delivery, pickup]",NaN,"{'address1': '7061 10th St N', 'address2': '',...",+16517357306,(651) 735-7306,23576.190524
